## Signature verification

In this exercise, you will write RIOT shell commands to sign and verify simple messages:
- One command that signs and print the signature
- One command that verify the signature
- One command to generate the key pair used to sign/verify the message

The signature mechanism is provided by an external package of RIOT, `c25519`.

The signature is stored in memory in a byte array and, to print it, it must be converted to a string of hexadecimal characters. RIOT provides a helper module, `fmt`, with useful functions to do that easily.

To generate random asymmetric public and secret keys, the application will use the `random` module of RIOT.

### Add required modules to the build

Since the application is about to implement shell functions, the `shell` module must be added to the build, as well as the `fmt` and the `random` modules. The `c25519` is provided as a package.

Edit the file Makefile and add there the required modules to the build.

```
USEMODULE += fmt
USEMODULE += shell
USEMODULE += random

USEPKG += c25519
```

### Implement the application

Now edit the `main.c` file as follows:

    
1. Add the required includes corresponding to the fmt, shell, random and c25519 modules that will be used by the application:

```c
#include "shell.h"
#include "fmt.h"
#include "edsign.h"
#include "ed25519.h"
#include "random.h"
```

2. Implement the key command handler function that will generate a new pair of key each time it is called:

    - First, under the header includes, declare 2 buffers where the asymmetric keys are stored:
    ```c
    static uint8_t secret_key[EDSIGN_SECRET_KEY_SIZE] = { 0 };
    static uint8_t public_key[EDSIGN_PUBLIC_KEY_SIZE] = { 0 };
    ```

    - In the `_key_handler` function, call the functions that will create the new keypair:
    ```c
    random_bytes(secret_key, sizeof(secret_key));
    ed25519_prepare(secret_key);
    edsign_sec_to_pub(public_key, secret_key);
    ```

3. Implement the sign command handler function that will generate a signature from an input message:

    - First, under the header includes, declare 2 buffers where the signatures (byte and hex representation) are stored:
    ```c
    static uint8_t signature[EDSIGN_SIGNATURE_SIZE] = { 0 };
    static char signature_hex[EDSIGN_SIGNATURE_SIZE * 2] = { 0 };
    ```

     **Note:** the size of the hexadecimal string signature buffer is twice as large as the signature buffer itself, because a single byte is represented by 2 hexadecimal characters.

    - In the `_sign_handler` function, call the functions that will generate the signature of the input command argument message in `argv[1]`:
    ```c
    edsign_sign(signature, public_key, secret_key, (uint8_t *)argv[1], strlen(argv[1]));
    ```

    - Convert the signature (a byte array) to its hexadecimal string representation:
    ```c
    fmt_bytes_hex(signature_hex, signature, EDSIGN_SIGNATURE_SIZE);
    ```

4. Implement the verify command handler function:
    The message and the signature are provided as command line argument respectively in `argv[1]` and `argv[2]`.

    - In the `_verify_handler` function, you must first convert the signature provided as a string of hex characters to a byte array, this is done with the `hex_bytes` function (from the `fmt` module):
    ```
    fmt_hex_bytes(signature, argv[2]);
    ```

    - Now call the functions that check the signature:
    ```c
    if (edsign_verify(signature, public_key, (uint8_t *)argv[1], strlen(argv[1]))) {
        puts("Message verified");
    }
    else {
        puts("Message not verified");
    }
    ```

### Build and test the application on native

Open a terminal with the `File > New > Terminal` menu and build for native (the default board, according to the `Makefile`):

Once the build is passing, start the native instance and play with the RIOT shell:

First, generate a new keypair:

Sign a first message:

Verify the signature:

### Build and test the application on IoT-LAB

The application is now ready and can be tested on real hardware, on an iotlab-m3 in the IoT-LAB testbed.

Choose your site (grenoble|lille|strasbourg):

In [ ]:
%env SITE=grenoble

1. Submit an experiment with one M3 device:

In [ ]:
!iotlab-experiment submit -n "riot-signature" -d 20 -l 1,archi=m3:at86rf231+site=$SITE

In [ ]:
!iotlab-experiment wait

In [ ]:
!iotlab-experiment get --nodes

2. Build and flash the shell application for the iotlab-m3 target device:

In [ ]:
!make BOARD=iotlab-m3 IOTLAB_NODE=auto-ssh flash

3. In the terminal, open the serial link:

Interact with the shell on the IoT-LAB M3 and verify that you can sign and verify signatures:

### Free up the resources

Stop your experiment to free up the device:

In [ ]:
!iotlab-experiment stop